## 1. 환경설정

In [1]:
import os
import time
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras import layers
from tensorflow import keras
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
cd /content/drive/MyDrive/Classroom/23-2_AI_Class/Assignment4

/content/drive/MyDrive/Classroom/23-2_AI_Class/Assignment4


In [4]:
ls

all_index_sentences_test.txt        all_word_pos_sentences_test.txt
all_index_sentences_train.txt       all_word_pos_sentences_train.txt
all_index_sentences_validation.txt  all_word_pos_sentences_validation.txt
all_word_pos_sentences_all.txt      assignment4.ipynb


## 2. 데이터준비

In [5]:
def build_vocabulary_temp(corpus_path):
    """
    단어와 그 출현횟수를 dict 타입의 사전에 수집한다.
    """
    Vocab_temporary = {}
    fp = open(corpus_path, "r", encoding="utf-8")
    for line in fp.readlines():
        sentence = line.split()
        if sentence[0] == '<<':
            continue
        for word_pos_pair in sentence:
            w_p = word_pos_pair.split('/')
            nseg = len(w_p)
            ## word/pos 내에 슬래시가 2개 이상 있어 3 조각 이상이 나옴.
            if nseg > 2:    ## 마지막 슬래시를 기준으로 단어와 품사로 구분함.
                word = ''
                for i in range(nseg-1):
                    word = word + w_p[i] + '/'
                word = word[:-1]    # remove the last slash.
            else:
                word = w_p[0]

            pos = w_p[-1]   # the last segment
            if word in Vocab_temporary:
                Vocab_temporary[word] += 1 # value값을 1 증가시키기
            else:
                Vocab_temporary[word] = 1
    fp.close()
    return Vocab_temporary

# 1 차적인 임시사전을 만든다. 사전명: Vocab_temporary
# key: 단어, value:출현횟수
Vocab_temporary = build_vocabulary_temp("./all_word_pos_sentences_all.txt")

In [6]:
# 사전 단어들을 출현횟수로 내림차순으로 정렬하여 그 결과를 리스트로 받는다.
sorted_Vocab = sorted(Vocab_temporary.items(), key = lambda kv: kv[1], reverse=True)
Total_n_words = len(sorted_Vocab)
print('파일에서 모은 총 단어수:', Total_n_words)
print(f'sorted_Vocab의 일부 내용 출력: {sorted_Vocab[0:10]}')

파일에서 모은 총 단어수: 51457
sorted_Vocab의 일부 내용 출력: [(',', 66321), ('the', 56317), ('.', 52726), ('of', 31003), ('to', 29976), ('a', 25848), ('and', 21632), ('in', 20765), ("'s", 12661), ('that', 10921)]


In [7]:
# 두 개의 특수단어('[PAD]', '[UNK]')를 포함하는 정식 사전(사전명: Vocab) 을 만든다:
# key: 단어, value: 단어번호
Vocab = {}
Vocab['[PAD]'] = 0  # Padding을 의미하는 특수단어 추가
Vocab['[UNK]'] = 1  # Unkown을 의미하는 특수단어 추가

# Vocab_temporary 에 모은 단어들에게는 단어번호를 2 부터 준다.
for i in range(Total_n_words):
    word = sorted_Vocab[i][0]
    freq = sorted_Vocab[i][1]
    Vocab[word] = i + 2

Total_number_words = len(Vocab)
print("최종 사전의 총 단어수:", Total_number_words)

최종 사전의 총 단어수: 51459


In [8]:
# 역-단어사전 i_Vocab  만들기:  Vocab 에서 key와 value 를 바꾼 사전.
# key: 단어번호, value: 단어
all_wps = list(Vocab.keys())
i_Vocab = {}
for word in all_wps:
  widx = Vocab[word]
  i_Vocab[widx] = word

In [9]:
# 품사 사전 (사전명: dic_POS)만들기:  key:품사명,  value: 품사번호
# 두 특수단어에게는 품사명/품사번호를  '[PAD]': 0,     '[UNK]': 1 로 준다.
# 나머지는 penn-tree-bank 의 48 개의 품사들에게 품사 번호를 2 부터 부여한다(2~49)
# 결국 총 품사는 총 50 개로 번호는 0 ~ 49 가 된다.
all_pos_list = ['[PAD]', '[UNK]',
                'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS',
                'MD', 'NN', 'NNS', 'NNP', 'NNPS', 'PDT', 'POS', 'PRP', 'PRP$', 'RB',
                'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN',
                'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB', '#', '$', '.', ',',
                ':', '(', ')', '\'\'', '\'', '``', '&rsquo', '”']
dic_POS = {}
for i in range(len(all_pos_list)):
    dic_POS[all_pos_list[i]] = i

num_pos = len(dic_POS)
print("총 품사수: ", num_pos)

총 품사수:  50


In [10]:
# 역-품사사전 만들기: dic_POS 에서 key와 value 를 바꾼 사전.
# key: 품사번호, value: 품사명
all_pos = list(dic_POS.keys())
i_dic_POS = {}
for a_pos in all_pos:
  pidx = dic_POS[a_pos]
  i_dic_POS[pidx] = a_pos

In [11]:
# 단어/품사명으로된 파일로부터 단어번호/품사번호으로된 파일을 생성한다
def build_index_sentences(path_word_pos_sentence_file, path_index_sentence_file):
    fp = open(path_word_pos_sentence_file, "r", encoding="utf-8")
    fp_w = open(path_index_sentence_file, "w", encoding="utf-8")

    for line in fp.readlines():
        sentence = line.split()
        if sentence[0] == '<<':   ## 파일명을 가지는 줄은 무시한다.
            continue

        line_widx = ''  # line for word indices
        line_pidx = ''  # line for pos indices

        for word_pos_pair in sentence:
            w_p = word_pos_pair.split('/')
            nseg = len(w_p)
            if nseg > 2:
                word = ''
                for i in range(nseg-1):
                    word = word + w_p[i] + '/'
                word = word[:-1]    # remove the last slash.
            else:
                word = w_p[0]

            pos = w_p[-1]   # the last segment
            if not(word in Vocab):  # Other scheme: Vocab.get(word) 가 None 이면 없는 것을 말함.
                widx = 1    # give index of [UNK] since it is missing in Vocab.
            else:
                widx = Vocab[word]

            if not(pos in dic_POS):
                pos_list = pos.split('|')
                pos = pos_list[-1]
                if not (pos in dic_POS):
                    print("exception occurs at dic_POS look_up. w_p=", w_p, " pos=", pos)
                    time.sleep(100)
                else:
                    pidx = dic_POS[pos]
            else:
                pidx = dic_POS[pos]

            if len(line_widx) == 0:
                line_widx = line_widx + str(widx)
            else:
                line_widx = line_widx + '\t' + str(widx)

            if len(line_pidx) == 0:
                line_pidx = line_pidx + str(pidx)
            else:
                line_pidx = line_pidx + '\t' + str(pidx)

        fp_w.write(line_widx + '\n')
        fp_w.write(line_pidx + '\n')
        fp_w.write('\n')    # an empty line after each sentence
    fp_w.close()
    fp.close()

build_index_sentences("./all_word_pos_sentences_train.txt", "./all_index_sentences_train.txt")
build_index_sentences("./all_word_pos_sentences_validation.txt", "./all_index_sentences_validation.txt")
build_index_sentences("./all_word_pos_sentences_test.txt", "./all_index_sentences_test.txt")

In [12]:
# 훈련예제 준비 함수
# 파일경로를 입력으로 받는다. 이 파일은 all_index_sentences_???.txt 이다.
# 출력: 파일내의 모든 문장들에 대한 정보를 이용하여 다음을 준비하여 출력한다.
#   1) list_X: 문장들의 단어 번호 리스트를 원소로 가지는 리스트
#   2) list_Y: 문장들의 정답품사번호 리스트를 원소로 가지는 리스트
#   3) list_leng: 문장들의 길이를 원소로 가지는 리스트

# Hyperparamter 정의
Max_seq_length = 128    ## time step 수 (MSL)

def load_X_and_Y(path_index_file):
    fp= open(path_index_file, "r", encoding="utf-8")
    list_X = []
    list_Y = []
    list_leng = []

    while True:
        # read two lines
        wordline = fp.readline()
        line_leng = len(wordline)

        if line_leng == 0:
            break   # end of file has come.
        if line_leng == 1:
            continue    # empty line used as sentence delimeter

        # The line read just before is a line of word indices.
        # The next line should be the corresponding pos index line.
        posline = fp.readline()
        w_index = wordline.split()
        p_index = posline.split()

        # X : a list of indices of words in a sentence.
        # Y : a list of pos indices of words in the sentence of X.
        X = []
        Y = []

        leng = len(w_index)
        if leng > Max_seq_length:
            leng = Max_seq_length   # truncation is done.

        for i in range(leng):
            X.append(int(w_index[i]))
            Y.append(int(p_index[i]))

        # pads are added after sentence
        if leng < Max_seq_length:
            for i in range(leng, Max_seq_length):
                X.append(0)     # word index of '[PAD]' which is 0 is added.
                Y.append(0)     # pos index of  '[PAD]' which is 0 is added.

        list_X.append(X)
        list_Y.append(Y)
        list_leng.append(leng)

    fp.close()
    return list_X, list_Y, list_leng

# Training 데이터
x_train, y_train, leng_train = load_X_and_Y("./all_index_sentences_train.txt")
x_train = np.array(x_train, dtype='i')
y_train = np.array(y_train, dtype='i')

# Validation 데이터
x_validation, y_validation, leng_valiation = load_X_and_Y("./all_index_sentences_validation.txt")
x_validation = np.array(x_validation, dtype='i')
y_validation = np.array(y_validation, dtype='i')

# Test 데이터
x_test, y_test, leng_test = load_X_and_Y("./all_index_sentences_test.txt")
x_test = np.array(x_test, dtype='i')
y_test = np.array(y_test, dtype='i')

## 3. 모델 정의

In [13]:
# 상수 선언
Vocab_size = 51459	# 사전 Vocab 의 총 단어수
Num_POS = 50    ## 총 품사 수

# Hyperparamter 정의
d_embed = 100     ## word embedding vector 의 길이 (원소수)

# Model 설계
model = tf.keras.models.Sequential()
# 층0:  word-embedding 층
model.add(tf.keras.layers.Embedding(Vocab_size, d_embed, embeddings_initializer='random_normal', input_length=Max_seq_length, mask_zero=True, trainable=True))	# output shape: (bsz, MSL, d_emb)
# 충1: LSTM 층
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences=True), input_shape=(Max_seq_length, d_embed)))
# output shape: (batch_sz, MSL, 512)
# 충2: LSTM 층
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True), input_shape=(Max_seq_length, 512)))
# output shape: (batch_sz, MSL, 256)
# 충3: LSTM 층
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True), input_shape=(Max_seq_length, 256)))
# output shape: (batch_sz, MSL, 128)
# 층4: NN 층
model.add(tf.keras.layers.Dense(units=Num_POS, activation='softmax', use_bias=True))	# 최종출력층. 각 시간의 각 단어마다 num_POS=50개의 확률이 생성됨.
# output shape: (batch_sz, MSL, num_POS)

## 4. 학습

In [14]:
# Hyperparamter 정의
LEARNING_RATE = 0.7e-4
BATCH_SIZE = 64
EPOCHS = 20

optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=['acc'])
model.fit(x=x_train, y=y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(x_validation, y_validation), shuffle=True, verbose=1)

Epoch 1/20
844/844 [==============================] - 117s 102ms/step - loss: 2.9236 - acc: 0.1831 - val_loss: 2.7986 - val_acc: 0.1974
Epoch 2/20
844/844 [==============================] - 46s 55ms/step - loss: 2.7475 - acc: 0.2049 - val_loss: 2.7203 - val_acc: 0.2078
Epoch 3/20
844/844 [==============================] - 42s 50ms/step - loss: 2.4823 - acc: 0.3114 - val_loss: 2.0908 - val_acc: 0.4303
Epoch 4/20
844/844 [==============================] - 43s 51ms/step - loss: 1.6605 - acc: 0.5383 - val_loss: 1.3132 - val_acc: 0.6416
Epoch 5/20
844/844 [==============================] - 38s 44ms/step - loss: 1.0161 - acc: 0.7456 - val_loss: 0.8375 - val_acc: 0.8010
Epoch 6/20
844/844 [==============================] - 42s 49ms/step - loss: 0.6464 - acc: 0.8581 - val_loss: 0.5862 - val_acc: 0.8707
Epoch 7/20
844/844 [==============================] - 42s 50ms/step - loss: 0.4445 - acc: 0.9042 - val_loss: 0.4482 - val_acc: 0.8972
Epoch 8/20
844/844 [==============================] - 39s 47

## 5. 추론

In [15]:
pred = model.predict(x=x_test, verbose=1)   # batch 단위로  수행함 (default = 32)    # batch 별 결과를 모아서 전체 결과를 반환함
pred_label = tf.math.argmax(pred, axis=2)
pred_label = pred_label.numpy()     # this is of 2-dimension (num_test_sents, msl)
num_test_sentences = y_test.shape[0]  # y_test has a shape of (num_test_sents, msl)

240/240 [==============================] - 12s 18ms/step


## 6. 결과 분석

In [16]:
num_correct = 0

for i in range(len(leng_test)):
    for j in range(leng_test[i]):
        if y_test[i][j] == pred_label[i][j]:
            num_correct += 1

acc = num_correct/sum(leng_test)*100

In [17]:
print("test accuracy = ", acc)
print("Program ends.")

test accuracy =  94.88637145728089
Program ends.


In [20]:
for i in range(20):
    print(f'({i+1}) ', end="")
    for j in range(leng_test[i]):
        print(i_Vocab[x_test[i][j]]+ "/"+i_dic_POS[pred_label[i][j]],end="")
        if y_test[i][j] != pred_label[i][j]:
            print("<"+ i_dic_POS[y_test[i][j]] + ">",end="")
        print(" ", end="")
    print("")

(1) Arthur/NNP M./NNP Goldberg/NNP said/VBD he/PRP extended/VBD his/PRP$ unsolicited/JJ tender/NN offer/NN of/IN $/$ 32/CD 
(2) a/DT share/NN tender/NN offer/NN ,/, or/CC $/$ 154.3/NN<CD> million/CD ,/, for/IN Di/NNP Giorgio/NNP Corp./NNP to/TO Nov./NNP 1/CD ./. 
(3) DIG/NNP Acquisition/NNP Corp./NNP ,/, the/DT New/NNP Jersey/NNP investor/NN 's/POS acquisition/NN vehicle/NN ,/, said/VBD that/IN as/IN of/IN the/DT close/NN of/IN business/NN yesterday/NN ,/, 560,839/VBG<CD> shares/NNS had/VBD been/VBN tendered/VBN ./. 
(4) Including/VBG the/DT stake/NN DIG/NNP already/RB held/VBN<VBD> ,/, DIG/NNP holds/VBZ a/DT total/NN of/IN about/RB 25/CD %/NN of/IN Di/NNP Giorgio/NNP 's/POS shares/NNS on/IN a/DT fully/RB diluted/VBN basis/NN ./. 
(5) The/DT offer/NN ,/, which/WDT also/RB includes/VBZ common/JJ and/CC preferred/JJ<VBN> stock/NN purchase/NN rights/NNS ,/, was/VBD to/TO expire/VB last/JJ night/NN at/IN midnight/NN ./. 
(6) The/DT new/JJ expiration/NN date/NN is/VBZ the/DT date/NN on/IN w